In [ ]:
# Missing Values Handling in CSV Datasets
# Complete Jupyter Notebook with Functions, Examples, and Comparisons

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("Missing Values Handler - Comprehensive Analysis")
print("=" * 50)

# ============================================================================
# SECTION 1: MISSING VALUES HANDLING FUNCTIONS
# ============================================================================

def drop_nulls(df):
    """
    Remove all rows containing at least one null value.
    
    Args:
        df (pd.DataFrame): Input DataFrame
        
    Returns:
        pd.DataFrame: DataFrame with null rows removed
        
    Pros:
        - Simple and straightforward
        - Preserves data integrity for remaining records
        - No assumptions about missing data
        
    Cons:
        - Can significantly reduce dataset size
        - May introduce bias if nulls are not random
        - Loss of potentially valuable information
    """
    return df.dropna()

def replace_with_fixed(df, value=0):
    """
    Replace all nulls with a user-defined value.
    
    Args:
        df (pd.DataFrame): Input DataFrame
        value: Fixed value to replace nulls (default: 0)
        
    Returns:
        pd.DataFrame: DataFrame with nulls replaced by fixed value
        
    Pros:
        - Simple implementation
        - Preserves dataset size
        - Works with any data type
        
    Cons:
        - May introduce artificial patterns
        - Fixed value might not be meaningful
        - Can skew statistical analysis
    """
    return df.fillna(value)

def replace_with_mean(df):
    """
    Replace nulls in numeric columns with the column mean.
    
    Args:
        df (pd.DataFrame): Input DataFrame
        
    Returns:
        pd.DataFrame: DataFrame with numeric nulls replaced by mean
        
    Pros:
        - Preserves column mean
        - Reasonable for normally distributed data
        - Maintains dataset size
        
    Cons:
        - Reduces variance
        - Not suitable for skewed distributions
        - Only works with numeric data
    """
    df_filled = df.copy()
    numeric_cols = df_filled.select_dtypes(include=[np.number]).columns
    
    for col in numeric_cols:
        if df_filled[col].isnull().any():
            mean_val = df_filled[col].mean()
            df_filled[col].fillna(mean_val, inplace=True)
    
    return df_filled

def replace_with_median(df):
    """
    Replace nulls in numeric columns with the column median.
    
    Args:
        df (pd.DataFrame): Input DataFrame
        
    Returns:
        pd.DataFrame: DataFrame with numeric nulls replaced by median
        
    Pros:
        - Robust to outliers
        - Good for skewed distributions
        - Preserves central tendency
        
    Cons:
        - May not preserve mean
        - Reduces variance
        - Only works with numeric data
    """
    df_filled = df.copy()
    numeric_cols = df_filled.select_dtypes(include=[np.number]).columns
    
    for col in numeric_cols:
        if df_filled[col].isnull().any():
            median_val = df_filled[col].median()
            df_filled[col].fillna(median_val, inplace=True)
    
    return df_filled

def replace_with_mode(df):
    """
    Replace nulls with the most frequent value in each column.
    
    Args:
        df (pd.DataFrame): Input DataFrame
        
    Returns:
        pd.DataFrame: DataFrame with nulls replaced by mode
        
    Pros:
        - Works with categorical and numeric data
        - Preserves most common patterns
        - Logical for categorical variables
        
    Cons:
        - May not exist for continuous variables
        - Can introduce bias toward common values
        - Multiple modes create ambiguity
    """
    df_filled = df.copy()
    
    for col in df_filled.columns:
        if df_filled[col].isnull().any():
            mode_val = df_filled[col].mode()
            if not mode_val.empty:
                df_filled[col].fillna(mode_val.iloc[0], inplace=True)
            else:
                # If no mode exists, use a default based on data type
                if df_filled[col].dtype in ['object', 'category']:
                    df_filled[col].fillna('Unknown', inplace=True)
                else:
                    df_filled[col].fillna(0, inplace=True)
    
    return df_filled

def forward_fill(df):
    """
    Fill nulls with the previous non-null value (forward fill).
    
    Args:
        df (pd.DataFrame): Input DataFrame
        
    Returns:
        pd.DataFrame: DataFrame with forward-filled values
        
    Pros:
        - Maintains temporal continuity
        - Good for time series data
        - Preserves trends
        
    Cons:
        - Assumes values persist over time
        - First rows may remain null
        - May propagate outdated information
    """
    return df.fillna(method='ffill')

def backward_fill(df):
    """
    Fill nulls with the next non-null value (backward fill).
    
    Args:
        df (pd.DataFrame): Input DataFrame
        
    Returns:
        pd.DataFrame: DataFrame with backward-filled values
        
    Pros:
        - Good for time series preparation
        - Fills early missing values
        - Maintains data relationships
        
    Cons:
        - Uses future information
        - Last rows may remain null
        - May not be logical for some contexts
    """
    return df.fillna(method='bfill')

# ============================================================================
# SECTION 2: SAMPLE DATA CREATION
# ============================================================================

# Load sample dataset from CSV instead of creating synthetic data
print("Loading sample dataset from 's.csv'...")
sample_df = pd.read_csv('sample_csv/sample.csv')

print("Sample dataset loaded from 'sample_csv/sample.csv'")

# ============================================================================
# SECTION 3: DATASET OVERVIEW AND ANALYSIS
# ============================================================================

print("\n" + "="*50)
print("DATASET OVERVIEW")
print("="*50)

print(f"Dataset shape: {sample_df.shape}")
print(f"Total cells: {sample_df.size}")
print(f"Missing values: {sample_df.isnull().sum().sum()}")
print(f"Missing percentage: {(sample_df.isnull().sum().sum() / sample_df.size) * 100:.2f}%")

print("\nMissing values by column:")
missing_summary = pd.DataFrame({
    'Column': sample_df.columns,
    'Missing_Count': sample_df.isnull().sum().values,
    'Missing_Percentage': (sample_df.isnull().sum().values / len(sample_df)) * 100,
    'Data_Type': sample_df.dtypes.values
})
print(missing_summary)

print("\nFirst 10 rows of sample data:")
print(sample_df.head(10))

# ============================================================================
# SECTION 4: APPLYING MISSING VALUE HANDLING METHODS
# ============================================================================

print("\n" + "="*50)
print("APPLYING MISSING VALUE HANDLING METHODS")
print("="*50)

# Dictionary to store results
results = {}
methods = {
    'Original': sample_df,
    'Drop Nulls': drop_nulls(sample_df),
    'Fixed Value (0)': replace_with_fixed(sample_df, 0),
    'Fixed Value (Unknown)': replace_with_fixed(sample_df, 'Unknown'),
    'Mean Fill': replace_with_mean(sample_df),
    'Median Fill': replace_with_median(sample_df),
    'Mode Fill': replace_with_mode(sample_df),
    'Forward Fill': forward_fill(sample_df),
    'Backward Fill': backward_fill(sample_df)
}

# Apply each method and collect statistics
for method_name, df_result in methods.items():
    total_nulls = df_result.isnull().sum().sum()
    shape = df_result.shape
    results[method_name] = {
        'dataframe': df_result,
        'total_nulls': total_nulls,
        'shape': shape,
        'null_percentage': (total_nulls / df_result.size) * 100 if df_result.size > 0 else 0
    }

# Display results for each method
for method_name, result in results.items():
    print(f"\n{method_name}:")
    print(f"  Shape: {result['shape']}")
    print(f"  Total nulls: {result['total_nulls']}")
    print(f"  Null percentage: {result['null_percentage']:.2f}%")
    
    if method_name != 'Original':
        print("  First 5 rows:")
        print(result['dataframe'].head().to_string(index=False))

# ============================================================================
# SECTION 5: DETAILED COMPARISON AND VISUALIZATION
# ============================================================================

print("\n" + "="*50)
print("DETAILED COMPARISON AND VISUALIZATION")
print("="*50)

# Create comparison dataframe
comparison_df = pd.DataFrame({
    'Method': list(results.keys()),
    'Rows': [results[method]['shape'][0] for method in results.keys()],
    'Columns': [results[method]['shape'][1] for method in results.keys()],
    'Total_Nulls': [results[method]['total_nulls'] for method in results.keys()],
    'Null_Percentage': [results[method]['null_percentage'] for method in results.keys()]
})

print("Comparison Summary:")
print(comparison_df.to_string(index=False))

# Visualization
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# 1. Bar chart of remaining nulls
axes[0, 0].bar(comparison_df['Method'], comparison_df['Total_Nulls'], color='skyblue')
axes[0, 0].set_title('Total Null Values by Method')
axes[0, 0].set_xlabel('Method')
axes[0, 0].set_ylabel('Number of Nulls')
axes[0, 0].tick_params(axis='x', rotation=45)

# 2. Bar chart of dataset sizes
axes[0, 1].bar(comparison_df['Method'], comparison_df['Rows'], color='lightcoral')
axes[0, 1].set_title('Dataset Size (Rows) by Method')
axes[0, 1].set_xlabel('Method')
axes[0, 1].set_ylabel('Number of Rows')
axes[0, 1].tick_params(axis='x', rotation=45)

# 3. Null percentage
axes[1, 0].bar(comparison_df['Method'], comparison_df['Null_Percentage'], color='lightgreen')
axes[1, 0].set_title('Null Percentage by Method')
axes[1, 0].set_xlabel('Method')
axes[1, 0].set_ylabel('Null Percentage (%)')
axes[1, 0].tick_params(axis='x', rotation=45)

# 4. Heatmap of nulls by column for original data
null_by_column = sample_df.isnull().sum()
axes[1, 1].bar(range(len(null_by_column)), null_by_column.values, color='orange')
axes[1, 1].set_title('Null Values by Column (Original Data)')
axes[1, 1].set_xlabel('Column Index')
axes[1, 1].set_ylabel('Number of Nulls')
axes[1, 1].set_xticks(range(len(null_by_column)))
axes[1, 1].set_xticklabels([col[:8] + '...' if len(col) > 8 else col 
                           for col in null_by_column.index], rotation=45)

plt.tight_layout()
plt.show()

# ============================================================================
# SECTION 6: EDGE CASE DEMONSTRATIONS
# ============================================================================

print("\n" + "="*50)
print("EDGE CASE DEMONSTRATIONS")
print("="*50)

# Edge Case 1: All nulls column
print("Edge Case 1: Column with all null values")
print("Original 'all_nulls' column unique values:", sample_df['all_null_column'].unique())

print("\nAfter mode fill:")
mode_filled = replace_with_mode(sample_df)
print("Mode-filled 'all_nulls' column unique values:", mode_filled['all_null_column'].unique())

print("\nAfter mean fill:")
mean_filled = replace_with_mean(sample_df)
print("Mean-filled 'all_nulls' column (still null):", mean_filled['all_null_column'].isnull().all())

# Edge Case 2: Mixed-type column
print("\nEdge Case 2: Mixed-type column handling")
print("Original 'mixed_type' column sample values:")
print(sample_df['mixed_type'].dropna().head(10).tolist())

print("\nAfter mode fill (mixed-type):")
print("Mode-filled 'mixed_type' column sample values:")
print(mode_filled['mixed_type'].head(10).tolist())


# Edge Case 3: Small dataset edge case
print("\nEdge Case 4: Very small dataset")
tiny_df = pd.DataFrame({
    'A': [1, np.nan, 3],
    'B': [np.nan, 5, np.nan],
    'C': ['x', np.nan, 'z']
})
print("Tiny dataset:")
print(tiny_df)

print("\nAfter dropping nulls (tiny dataset):")
tiny_dropped = drop_nulls(tiny_df)
print(tiny_dropped)
print(f"Remaining rows: {len(tiny_dropped)}")

# ============================================================================
# SECTION 7: PERFORMANCE AND MEMORY CONSIDERATIONS
# ============================================================================

print("\n" + "="*50)
print("PERFORMANCE AND MEMORY CONSIDERATIONS")
print("="*50)

import time
import psutil
import os

def measure_performance(func, df, *args):
    """Measure execution time and memory usage of a function."""
    process = psutil.Process(os.getpid())
    mem_before = process.memory_info().rss / 1024 / 1024  # MB
    
    start_time = time.time()
    result = func(df, *args)
    end_time = time.time()
    
    mem_after = process.memory_info().rss / 1024 / 1024  # MB
    
    return {
        'execution_time': end_time - start_time,
        'memory_delta': mem_after - mem_before,
        'result_shape': result.shape
    }

# Create a larger dataset for performance testing
print("Creating larger dataset for performance testing...")
large_df = sample_df
# Replicate to make it larger
large_df = pd.concat([large_df] * 10, ignore_index=True)

print(f"Large dataset shape: {large_df.shape}")

# Test performance of each method
performance_results = {}
methods_to_test = {
    'drop_nulls': drop_nulls,
    'replace_with_fixed': lambda df: replace_with_fixed(df, 0),
    'replace_with_mean': replace_with_mean,
    'replace_with_median': replace_with_median,
    'replace_with_mode': replace_with_mode,
    'forward_fill': forward_fill,
    'backward_fill': backward_fill
}

for method_name, method_func in methods_to_test.items():
    perf = measure_performance(method_func, large_df)
    performance_results[method_name] = perf
    print(f"{method_name:20} | Time: {perf['execution_time']:.4f}s | "
          f"Memory: {perf['memory_delta']:+.2f}MB | "
          f"Result shape: {perf['result_shape']}")

# ============================================================================
# SECTION 8: RECOMMENDATIONS AND BEST PRACTICES
# ============================================================================

print("\n" + "="*50)
print("RECOMMENDATIONS AND BEST PRACTICES")
print("="*50)

recommendations = """
CHOOSING THE RIGHT METHOD FOR MISSING VALUES:

1. **Drop Nulls** - Use when:
   - Dataset is large and missing data is minimal (<5%)
   - Missing data appears to be random
   - Data quality is more important than quantity

2. **Fixed Value Replacement** - Use when:
   - Missing values have a meaningful default (e.g., 0 for counts)
   - Categorical data with clear "Unknown" category
   - Simple imputation is acceptable

3. **Mean/Median Fill** - Use when:
   - Numeric data with normal/skewed distribution respectively
   - Missing values are random
   - Preserving central tendency is important

4. **Mode Fill** - Use when:
   - Categorical data
   - Preserving most common patterns
   - Mixed data types

5. **Forward/Backward Fill** - Use when:
   - Time series data
   - Temporal continuity is important
   - Values are expected to persist over time

EDGE CASE HANDLING:
- Always check for columns with all nulls
- Handle mixed data types before imputation
- Consider the domain context when choosing methods
- Validate results after imputation

PERFORMANCE CONSIDERATIONS:
- Mode calculation is typically slowest
- Forward/backward fill are fastest for time series
- Memory usage varies by method complexity
"""

print(recommendations)

# ============================================================================
# SECTION 9: SAVE RESULTS TO FILES
# ============================================================================

print("\n" + "="*50)
print("SAVING RESULTS")
print("="*50)

# Save comparison results
comparison_df.to_csv('missing_values_comparison.csv', index=False)
print("Comparison results saved to 'missing_values_comparison.csv'")

# Save example of each method
for method_name, result in results.items():
    if method_name != 'Original':
        filename = f"result_{method_name.lower().replace(' ', '_').replace('(', '').replace(')', '')}.csv"
        result['dataframe'].to_csv(filename, index=False)
        print(f"{method_name} result saved to '{filename}'")

print("\n" + "="*50)
print("ANALYSIS COMPLETE")
print("="*50)
print("All missing value handling methods have been demonstrated and compared.")
print("Check the generated CSV files for detailed results.")
print("Consider the recommendations above when choosing a method for your specific use case")